# TextVibCLIP Text-Vibration 정렬 문제 분석 및 해결

**심각한 문제 발견:**
- Text Encoder 단독: 83.0% (우수)
- Vibration Encoder 단독: 88.6% (우수)  
- TextVibCLIP 통합: 14.4% (랜덤 수준)

**71.4%p 성능 손실의 원인을 찾아 해결하자!**


In [1]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.manifold import TSNE
from collections import Counter
import logging
import seaborn as sns
from torch.utils.data import DataLoader

# 프로젝트 루트 추가
sys.path.append('/data/home/kyj2024/TextVibCLIP')

from src.text_encoder import create_text_encoder
from src.vibration_encoder import VibrationEncoder
from src.textvib_model import TextVibCLIP
from src.data_loader import BearingDataset, create_collate_fn
from configs.model_config import MODEL_CONFIG

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🔍 Text-Vibration 정렬 문제 분석 시작")
print("=" * 60)


/data/home/kyj2024/anaconda3/envs/TVCLIP/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 Text-Vibration 정렬 문제 분석 시작


/data/home/kyj2024/anaconda3/envs/TVCLIP/lib/python3.8/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/data/home/kyj2024/anaconda3/envs/TVCLIP/lib/python3.8/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Plea

## 1. 데이터 및 모델 준비


In [2]:
# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"디바이스: {device}")

# UOS 데이터셋 로드
train_dataset = BearingDataset(
    data_dir='data_scenario1',
    dataset_type='uos',
    domain_value=600,
    subset='train'
)

test_dataset = BearingDataset(
    data_dir='data_scenario1',
    dataset_type='uos',
    domain_value=600,
    subset='test'
)

print(f"Train 데이터: {len(train_dataset)}개 샘플")
print(f"Test 데이터: {len(test_dataset)}개 샘플")

# 샘플 데이터 확인
sample = train_dataset[0]
print(f"\n샘플 구조:")
print(f"  라벨: {sample['labels']}")
print(f"  진동 신호 shape: {sample['vibration'].shape}")
print(f"  텍스트: {sample['text'][:100]}...")


INFO:src.data_loader:UOS 데이터 라벨 분포: {'B_6204': 1, 'H_6204': 4, 'IR_6204': 1, 'OR_6204': 1}
INFO:src.data_loader:최소 샘플 수: 1
INFO:src.data_loader:UOS Domain-Incremental 윈도우 레벨 분할:
INFO:src.data_loader:  모든 subset에 모든 7개 파일 포함
INFO:src.data_loader:  각 파일 내에서 윈도우 분할: Train 60%, Val 20%, Test 20%
INFO:src.data_loader:  Deep Groove Ball 7-클래스 분포: {'B': 1, 'H': 1, 'IR': 1, 'OR': 1, 'L': 1, 'M': 1, 'U': 1}
INFO:src.data_loader:  클래스 수: 7개 (균형 확인)
INFO:src.data_loader:  ✅ 완벽한 클래스 균형 달성!
INFO:src.data_loader:UOS train 분할 결과:
INFO:src.data_loader:  Train: 7개 파일, Val: 7개 파일, Test: 7개 파일


디바이스: cuda


INFO:src.data_loader:BearingDataset 초기화 완료 (UOS): 7개 파일, 1249개 윈도우/파일, 총 8743개 샘플, Domain: 600, Subset: train
INFO:src.data_loader:UOS 데이터 라벨 분포: {'B_6204': 1, 'H_6204': 4, 'IR_6204': 1, 'OR_6204': 1}
INFO:src.data_loader:최소 샘플 수: 1
INFO:src.data_loader:UOS Domain-Incremental 윈도우 레벨 분할:
INFO:src.data_loader:  모든 subset에 모든 7개 파일 포함
INFO:src.data_loader:  각 파일 내에서 윈도우 분할: Train 60%, Val 20%, Test 20%
INFO:src.data_loader:  Deep Groove Ball 7-클래스 분포: {'B': 1, 'H': 1, 'IR': 1, 'OR': 1, 'L': 1, 'M': 1, 'U': 1}
INFO:src.data_loader:  클래스 수: 7개 (균형 확인)
INFO:src.data_loader:  ✅ 완벽한 클래스 균형 달성!
INFO:src.data_loader:UOS test 분할 결과:
INFO:src.data_loader:  Train: 7개 파일, Val: 7개 파일, Test: 7개 파일
INFO:src.data_loader:BearingDataset 초기화 완료 (UOS): 7개 파일, 1249개 윈도우/파일, 총 8743개 샘플, Domain: 600, Subset: test
INFO:src.data_loader:인덱스 매핑 생성 완료: 8743개 (파일 7개 × 윈도우 1249개)


Train 데이터: 8743개 샘플
Test 데이터: 8743개 샘플

샘플 구조:
  라벨: tensor([6, 0])
  진동 신호 shape: torch.Size([2048])
  텍스트: Shaft misalignment detected...


## 2. 개별 Encoder 및 통합 모델 생성


In [3]:
# 1. 개별 Text Encoder 생성
text_encoder = create_text_encoder('first_domain')
text_encoder.to(device)
text_encoder.eval()

# 2. 개별 Vibration Encoder 생성
vibration_encoder = VibrationEncoder(
    input_length=MODEL_CONFIG['vibration_encoder']['input_length'],
    embedding_dim=MODEL_CONFIG['embedding_dim']
)
vibration_encoder.to(device)
vibration_encoder.eval()

# 3. TextVibCLIP 통합 모델 생성 (올바른 생성자 사용)
textvib_model = TextVibCLIP(
    domain_stage='first_domain',
    embedding_dim=MODEL_CONFIG['embedding_dim']
)
textvib_model.to(device)
textvib_model.eval()

print("🔧 모델 생성 완료:")
print(f"  개별 Text Encoder: {text_encoder.get_trainable_parameters():,}개 파라미터")
print(f"  개별 Vibration Encoder: {sum(p.numel() for p in vibration_encoder.parameters()):,}개 파라미터") 
print(f"  TextVibCLIP 통합: {sum(p.numel() for p in textvib_model.parameters()):,}개 파라미터")

# 임베딩 차원 확인
print(f"\n📏 임베딩 차원:")
print(f"  개별 Text Encoder → {MODEL_CONFIG['embedding_dim']}차원")
print(f"  개별 Vibration Encoder → {MODEL_CONFIG['embedding_dim']}차원")
print(f"  통합 모델 → {MODEL_CONFIG['embedding_dim']}차원")
print(f"  Projection → {MODEL_CONFIG['projection']['hidden_dim']}차원")

# 통합 모델의 내부 encoder 접근
print(f"\n🔍 통합 모델 구조:")
print(f"  통합 Text Encoder: {hasattr(textvib_model, 'text_encoder')}")
print(f"  통합 Vibration Encoder: {hasattr(textvib_model, 'vibration_encoder')}")
print(f"  InfoNCE Loss: {hasattr(textvib_model, 'infonce_loss')}")


INFO:src.text_encoder:LoRA 적용 완료: rank=32, alpha=64
INFO:src.text_encoder:Base DistilBERT 파라미터 freeze 완료: 100개 파라미터
INFO:src.text_encoder:TextEncoder 초기화 완료: LoRA=True, Freeze=True
INFO:src.text_encoder:TextEncoder 생성 (first_domain): Total=1,114,880, LoRA=589,824
INFO:src.vibration_encoder:1D-CNN VibrationEncoder 초기화: input_length=2048, embedding_dim=256
INFO:src.vibration_encoder:   OPTIMIZED: 커널 크기: [16, 32, 64, 32] - 4-layer 베어링 최적화
INFO:src.vibration_encoder:   OPTIMIZED: 채널 수: [64, 128, 256, 512] - 자연스러운 64→512 증가
INFO:src.vibration_encoder:   총 파라미터: 6,985,288
INFO:src.text_encoder:LoRA 적용 완료: rank=32, alpha=64
INFO:src.text_encoder:Base DistilBERT 파라미터 freeze 완료: 100개 파라미터
INFO:src.text_encoder:TextEncoder 초기화 완료: LoRA=True, Freeze=True
INFO:src.text_encoder:TextEncoder 생성 (first_domain): Total=1,114,880, LoRA=589,824
INFO:src.vibration_encoder:1D-CNN VibrationEncoder 초기화: input_length=2048, embedding_dim=256
INFO:src.vibration_encoder:   OPTIMIZED: 커널 크기: [16, 32, 64, 32] - 4-l

🔧 모델 생성 완료:
  개별 Text Encoder: 1,114,880개 파라미터
  개별 Vibration Encoder: 6,985,288개 파라미터
  TextVibCLIP 통합: 74,463,050개 파라미터

📏 임베딩 차원:
  개별 Text Encoder → 256차원
  개별 Vibration Encoder → 256차원
  통합 모델 → 256차원
  Projection → 512차원

🔍 통합 모델 구조:
  통합 Text Encoder: True
  통합 Vibration Encoder: True
  InfoNCE Loss: True


## 3. 임베딩 생성 및 정렬 상태 분석 (핵심)


In [4]:
# 테스트 데이터 샘플링 (각 클래스당 50개씩)
samples_per_class = 50
class_data = {i: {'texts': [], 'vibrations': [], 'labels': []} for i in range(7)}
class_counts = {i: 0 for i in range(7)}

print("🎯 클래스별 데이터 샘플링...")
for i in range(len(train_dataset)):
    if all(count >= samples_per_class for count in class_counts.values()):
        break
        
    sample = train_dataset[i]
    label = sample['labels'][0].item()
    
    if class_counts[label] < samples_per_class:
        class_data[label]['texts'].append(sample['text'])
        class_data[label]['vibrations'].append(sample['vibration'])
        class_data[label]['labels'].append(label)
        class_counts[label] += 1

print(f"샘플링 완료: {class_counts}")

# 데이터 결합
all_texts = []
all_vibrations = []
all_labels = []

for class_id in range(7):
    all_texts.extend(class_data[class_id]['texts'])
    all_vibrations.extend(class_data[class_id]['vibrations'])
    all_labels.extend(class_data[class_id]['labels'])

vibration_tensor = torch.stack(all_vibrations)
labels_tensor = torch.tensor(all_labels)

print(f"\n총 데이터: {len(all_texts)}개 (텍스트 + 진동)")
print(f"진동 텐서 shape: {vibration_tensor.shape}")
print(f"클래스 분포: {Counter(all_labels)}")


🎯 클래스별 데이터 샘플링...
샘플링 완료: {0: 50, 1: 50, 2: 50, 3: 50, 4: 50, 5: 50, 6: 50}

총 데이터: 350개 (텍스트 + 진동)
진동 텐서 shape: torch.Size([350, 2048])
클래스 분포: Counter({0: 50, 1: 50, 2: 50, 3: 50, 4: 50, 5: 50, 6: 50})


In [5]:
# 🔍 핵심: 개별 Encoder vs 통합 모델 임베딩 비교
print("\n🔍 임베딩 생성 및 비교 분석")
print("=" * 50)

batch_size = 32
with torch.no_grad():
    # 1. 개별 Text Encoder 임베딩
    text_embeddings_individual = text_encoder.encode_texts(all_texts, device)
    text_embeddings_individual = F.normalize(text_embeddings_individual, p=2, dim=1)
    
    # 2. 개별 Vibration Encoder 임베딩
    vib_embeddings_individual = []
    for i in range(0, len(vibration_tensor), batch_size):
        batch_vib = vibration_tensor[i:i+batch_size].to(device)
        batch_emb = vibration_encoder(batch_vib)
        batch_emb = F.normalize(batch_emb, p=2, dim=1)
        vib_embeddings_individual.append(batch_emb.cpu())
    vib_embeddings_individual = torch.cat(vib_embeddings_individual, dim=0)
    
    # 3. 통합 모델 임베딩 (TextVibCLIP) - 내부 encoder 직접 사용
    text_embeddings_integrated = []
    vib_embeddings_integrated = []
    
    for i in range(0, len(all_texts), batch_size):
        end_idx = min(i + batch_size, len(all_texts))
        batch_texts = all_texts[i:end_idx]
        batch_vibs = vibration_tensor[i:end_idx].to(device)
        
        # 통합 모델의 내부 encoder를 통해 임베딩 생성 (projection 없이)
        batch_text_emb = textvib_model.text_encoder.encode_texts(batch_texts, device)
        batch_vib_emb = textvib_model.vibration_encoder(batch_vibs)
        
        # 정규화 (TextVibCLIP과 동일한 방식)
        batch_text_emb = F.normalize(batch_text_emb, p=2, dim=1)
        batch_vib_emb = F.normalize(batch_vib_emb, p=2, dim=1)
        
        text_embeddings_integrated.append(batch_text_emb.cpu())
        vib_embeddings_integrated.append(batch_vib_emb.cpu())
    
    text_embeddings_integrated = torch.cat(text_embeddings_integrated, dim=0)
    vib_embeddings_integrated = torch.cat(vib_embeddings_integrated, dim=0)

print("✅ 임베딩 생성 완료")
print(f"  개별 Text: {text_embeddings_individual.shape}")
print(f"  개별 Vibration: {vib_embeddings_individual.shape}")
print(f"  통합 Text: {text_embeddings_integrated.shape}")
print(f"  통합 Vibration: {vib_embeddings_integrated.shape}")

# 임베딩 차원 및 동일성 확인 (디바이스 통일)
print(f"\n📐 임베딩 비교:")
print(f"  차원 일치: {text_embeddings_individual.shape == text_embeddings_integrated.shape}")

# 디바이스를 CPU로 통일하여 비교
text_individual_cpu = text_embeddings_individual.cpu()
vib_individual_cpu = vib_embeddings_individual.cpu()
text_integrated_cpu = text_embeddings_integrated.cpu()
vib_integrated_cpu = vib_embeddings_integrated.cpu()

text_diff = torch.norm(text_individual_cpu - text_integrated_cpu).item()
vib_diff = torch.norm(vib_individual_cpu - vib_integrated_cpu).item()

print(f"  개별 vs 통합 차이 (Text): {text_diff:.6f}")
print(f"  개별 vs 통합 차이 (Vibration): {vib_diff:.6f}")

# 동일성 검사
text_identical = torch.allclose(text_individual_cpu, text_integrated_cpu, atol=1e-6)
vib_identical = torch.allclose(vib_individual_cpu, vib_integrated_cpu, atol=1e-6)

print(f"  Text 임베딩 동일성: {text_identical}")
print(f"  Vibration 임베딩 동일성: {vib_identical}")

if text_identical and vib_identical:
    print("🚨 개별 모델과 통합 모델의 임베딩이 동일함!")
    print("   → Encoder 자체는 동일하게 작동")
    print("   → 문제는 InfoNCE Loss, Temperature, 또는 평가 로직에 있음")
elif text_diff < 1e-3 and vib_diff < 1e-3:
    print("⚠️ 개별과 통합 임베딩이 거의 동일함 (미세한 차이)")
    print("   → 수치적 정밀도 차이일 가능성")
else:
    print("❌ 개별과 통합 임베딩이 다름!")
    print("   → 통합 모델 내부에서 추가 변환이 발생")



🔍 임베딩 생성 및 비교 분석
✅ 임베딩 생성 완료
  개별 Text: torch.Size([350, 256])
  개별 Vibration: torch.Size([350, 256])
  통합 Text: torch.Size([350, 256])
  통합 Vibration: torch.Size([350, 256])

📐 임베딩 비교:
  차원 일치: True
  개별 vs 통합 차이 (Text): 25.719437
  개별 vs 통합 차이 (Vibration): 26.672676
  Text 임베딩 동일성: False
  Vibration 임베딩 동일성: False
❌ 개별과 통합 임베딩이 다름!
   → 통합 모델 내부에서 추가 변환이 발생


## 4. 정렬 상태 진단 (Critical Analysis)


In [6]:
# 🚨 핵심 분석: Text-Vibration 정렬 상태 진단
print("\n🚨 Text-Vibration 정렬 상태 진단")
print("=" * 60)

def analyze_alignment(text_emb, vib_emb, labels, title):
    """Text-Vibration 임베딩 정렬 상태 분석 (디바이스 안전)"""
    print(f"\n📊 {title}")
    print("-" * 40)
    
    # 🔧 디바이스 통일: 모든 텐서를 CPU로 이동
    text_emb_cpu = text_emb.cpu()
    vib_emb_cpu = vib_emb.cpu()
    labels_cpu = labels.cpu()
    
    # 1. 임베딩 통계
    print(f"Text 임베딩 통계:")
    print(f"  평균: {text_emb_cpu.mean().item():.6f}, 표준편차: {text_emb_cpu.std().item():.6f}")
    print(f"  범위: [{text_emb_cpu.min().item():.6f}, {text_emb_cpu.max().item():.6f}]")
    
    print(f"Vibration 임베딩 통계:")
    print(f"  평균: {vib_emb_cpu.mean().item():.6f}, 표준편차: {vib_emb_cpu.std().item():.6f}")
    print(f"  범위: [{vib_emb_cpu.min().item():.6f}, {vib_emb_cpu.max().item():.6f}]")
    
    # 2. Cross-modal 유사도 분석
    cross_similarity = torch.matmul(text_emb_cpu, vib_emb_cpu.t())  # (N, N)
    
    # 같은 클래스 간 유사도 (대각선)
    same_class_sim = torch.diag(cross_similarity).mean().item()
    
    # 다른 클래스 간 유사도 (비대각선)
    mask = ~torch.eye(len(labels_cpu), dtype=torch.bool)
    diff_class_sim = cross_similarity[mask].mean().item()
    
    alignment_score = same_class_sim - diff_class_sim
    
    print(f"Cross-modal 정렬 분석:")
    print(f"  같은 클래스 유사도: {same_class_sim:.6f}")
    print(f"  다른 클래스 유사도: {diff_class_sim:.6f}")
    print(f"  정렬 점수: {alignment_score:.6f} (클수록 좋음)")
    
    # 3. 클래스별 정렬 상태
    unique_labels = torch.unique(labels_cpu)
    print(f"클래스별 Text-Vibration 정렬:")
    
    class_alignments = []
    for cls in unique_labels:
        cls_mask = (labels_cpu == cls)
        cls_text = text_emb_cpu[cls_mask]
        cls_vib = vib_emb_cpu[cls_mask]
        
        # 클래스 내 Text-Vibration 유사도
        if len(cls_text) > 0:
            cls_sim = torch.matmul(cls_text, cls_vib.t()).diag().mean().item()
            class_alignments.append(cls_sim)
            print(f"  클래스 {cls.item()}: {cls_sim:.6f}")
    
    avg_class_alignment = np.mean(class_alignments)
    print(f"  평균 클래스 정렬: {avg_class_alignment:.6f}")
    
    return {
        'alignment_score': alignment_score,
        'same_class_sim': same_class_sim,
        'diff_class_sim': diff_class_sim,
        'avg_class_alignment': avg_class_alignment,
        'class_alignments': class_alignments
    }

# 개별 Encoder 정렬 분석
individual_analysis = analyze_alignment(
    text_embeddings_individual, 
    vib_embeddings_individual, 
    labels_tensor,
    "개별 Encoder 정렬 상태"
)

# 통합 모델 정렬 분석  
integrated_analysis = analyze_alignment(
    text_embeddings_integrated,
    vib_embeddings_integrated, 
    labels_tensor,
    "통합 모델 정렬 상태"
)



🚨 Text-Vibration 정렬 상태 진단

📊 개별 Encoder 정렬 상태
----------------------------------------
Text 임베딩 통계:
  평균: 0.002460, 표준편차: 0.062452
  범위: [-0.190191, 0.185345]
Vibration 임베딩 통계:
  평균: -0.001136, 표준편차: 0.062490
  범위: [-0.189785, 0.216230]
Cross-modal 정렬 분석:
  같은 클래스 유사도: 0.047532
  다른 클래스 유사도: 0.047442
  정렬 점수: 0.000090 (클수록 좋음)
클래스별 Text-Vibration 정렬:
  클래스 0: 0.054840
  클래스 1: 0.050084
  클래스 2: 0.049372
  클래스 3: 0.052083
  클래스 4: 0.042589
  클래스 5: 0.049161
  클래스 6: 0.034598
  평균 클래스 정렬: 0.047532

📊 통합 모델 정렬 상태
----------------------------------------
Text 임베딩 통계:
  평균: 0.004101, 표준편차: 0.062366
  범위: [-0.182250, 0.190655]
Vibration 임베딩 통계:
  평균: -0.002421, 표준편차: 0.062453
  범위: [-0.166377, 0.179656]
Cross-modal 정렬 분석:
  같은 클래스 유사도: 0.002629
  다른 클래스 유사도: 0.002651
  정렬 점수: -0.000022 (클수록 좋음)
클래스별 Text-Vibration 정렬:
  클래스 0: 0.003898
  클래스 1: 0.009290
  클래스 2: -0.004035
  클래스 3: 0.010970
  클래스 4: -0.017201
  클래스 5: 0.004601
  클래스 6: 0.010880
  평균 클래스 정렬: 0.002629


## 5. InfoNCE Loss 및 Temperature 분석


In [7]:
# 🔥 InfoNCE Loss 및 Temperature 파라미터 분석
print("\n🔥 InfoNCE Loss 분석")
print("=" * 50)

def calculate_infonce_loss(text_emb, vib_emb, temperature=0.07):
    """InfoNCE Loss 계산 및 분석 (디바이스 안전)"""
    # 🔧 디바이스 통일: 모든 텐서를 CPU로 이동
    text_emb_cpu = text_emb.cpu()
    vib_emb_cpu = vib_emb.cpu()
    
    # 유사도 행렬 계산
    similarity_matrix = torch.matmul(text_emb_cpu, vib_emb_cpu.t()) / temperature
    
    # 정답 레이블 (대각선)
    labels = torch.arange(len(text_emb_cpu))
    
    # Cross-entropy loss 계산
    loss = F.cross_entropy(similarity_matrix, labels)
    
    # 예측 정확도 계산
    predictions = torch.argmax(similarity_matrix, dim=1)
    accuracy = (predictions == labels).float().mean()
    
    return {
        'loss': loss.item(),
        'accuracy': accuracy.item(),
        'similarity_matrix': similarity_matrix,
        'predictions': predictions
    }

# 현재 설정된 Temperature 값들
temp_config = MODEL_CONFIG['infonce']
print(f"현재 Temperature 설정:")
print(f"  First domain text: {temp_config['first_domain_temperature_text']}")
print(f"  First domain vibration: {temp_config['first_domain_temperature_vib']}")
print(f"  Continual text: {temp_config['continual_temperature_text']}")
print(f"  Continual vibration: {temp_config['continual_temperature_vib']}")

# 다양한 Temperature 값으로 테스트
temperatures = [0.01, 0.05, 0.07, 0.1, 0.2, 0.5]
print(f"\n📈 Temperature별 InfoNCE 성능 (개별 Encoder):")
print("Temp\tLoss\tAccuracy")
print("-" * 30)

individual_results = []
for temp in temperatures:
    result = calculate_infonce_loss(
        text_embeddings_individual, 
        vib_embeddings_individual, 
        temperature=temp
    )
    individual_results.append((temp, result))
    print(f"{temp:.2f}\t{result['loss']:.4f}\t{result['accuracy']:.4f}")

print(f"\n📈 Temperature별 InfoNCE 성능 (통합 모델):")
print("Temp\tLoss\tAccuracy")
print("-" * 30)

integrated_results = []
for temp in temperatures:
    result = calculate_infonce_loss(
        text_embeddings_integrated,
        vib_embeddings_integrated, 
        temperature=temp
    )
    integrated_results.append((temp, result))
    print(f"{temp:.2f}\t{result['loss']:.4f}\t{result['accuracy']:.4f}")

# 최적 Temperature 찾기
best_individual = max(individual_results, key=lambda x: x[1]['accuracy'])
best_integrated = max(integrated_results, key=lambda x: x[1]['accuracy'])

print(f"\n🎯 최적 Temperature:")
print(f"  개별 Encoder: {best_individual[0]:.2f} (정확도: {best_individual[1]['accuracy']:.4f})")
print(f"  통합 모델: {best_integrated[0]:.2f} (정확도: {best_integrated[1]['accuracy']:.4f})")



🔥 InfoNCE Loss 분석
현재 Temperature 설정:
  First domain text: 0.05
  First domain vibration: 0.05
  Continual text: 0.07
  Continual vibration: 0.03

📈 Temperature별 InfoNCE 성능 (개별 Encoder):
Temp	Loss	Accuracy
------------------------------
0.01	5.9496	0.0029
0.05	5.8602	0.0029
0.07	5.8587	0.0029
0.10	5.8581	0.0029
0.20	5.8577	0.0029
0.50	5.8578	0.0029

📈 Temperature별 InfoNCE 성능 (통합 모델):
Temp	Loss	Accuracy
------------------------------
0.01	6.2571	0.0029
0.05	5.8786	0.0029
0.07	5.8688	0.0029
0.10	5.8634	0.0029
0.20	5.8594	0.0029
0.50	5.8582	0.0029

🎯 최적 Temperature:
  개별 Encoder: 0.01 (정확도: 0.0029)
  통합 모델: 0.01 (정확도: 0.0029)


## 6. 결론 및 해결 방안


In [8]:
print("\n" + "="*60)
print("🎯 Text-Vibration 정렬 문제 진단 결과")
print("="*60)

# 성능 비교 요약
print(f"📊 성능 요약:")
print(f"  개별 Text Encoder: 83.0% (이전 테스트)")
print(f"  개별 Vibration Encoder: 88.6% (이전 테스트)")
print(f"  TextVibCLIP 통합: 14.4% (실제 성능)")
print(f"  성능 손실: {83.0 + 88.6 - 14.4:.1f}%p (심각한 정렬 문제)")

# 정렬 분석 결과 요약
print(f"\n🔍 정렬 분석 결과:")
print(f"  개별 Encoder 정렬 점수: {individual_analysis['alignment_score']:.6f}")
print(f"  통합 모델 정렬 점수: {integrated_analysis['alignment_score']:.6f}")
print(f"  정렬 점수 차이: {individual_analysis['alignment_score'] - integrated_analysis['alignment_score']:.6f}")

# Temperature 분석 결과
print(f"\n🌡️ Temperature 최적화:")
print(f"  개별 최적 Temperature: {best_individual[0]:.2f}")
print(f"  통합 최적 Temperature: {best_integrated[0]:.2f}")
print(f"  현재 설정값: {temp_config['first_domain_temperature_text']:.2f}")

# 문제 진단 및 해결 방안
print(f"\n🚨 문제 진단:")
if integrated_analysis['alignment_score'] < individual_analysis['alignment_score'] * 0.5:
    print("  ❌ 심각한 정렬 문제: 통합 과정에서 정렬이 크게 악화됨")
    print("  원인: Projection Layer, InfoNCE Loss, 또는 모델 아키텍처 문제")
elif integrated_analysis['alignment_score'] < individual_analysis['alignment_score'] * 0.8:
    print("  ⚠️ 중간 정렬 문제: 통합 과정에서 일부 정렬 손실")
    print("  원인: Temperature 설정 또는 Loss 가중치 문제")
else:
    print("  ✅ 정렬 상태 양호: 다른 원인 탐색 필요")

print(f"\n💡 해결 방안 우선순위:")
print("1. 🔥 Temperature 최적화")
print(f"   - 현재: {temp_config['first_domain_temperature_text']:.2f}")
print(f"   - 권장: {best_integrated[0]:.2f}")

print("2. 🎯 Projection Layer 점검")
print("   - 차원 정렬 확인")
print("   - 정규화 방법 검토")

print("3. 📐 InfoNCE Loss 구현 검토")
print("   - 유사도 계산 로직")
print("   - Negative sampling 방식")

print("4. 🔧 모델 아키텍처 개선")
print("   - Cross-attention 메커니즘 추가")
print("   - Alignment regularization")

# 즉시 적용 가능한 수정사항
print(f"\n⚡ 즉시 수정 가능한 사항:")
if abs(best_integrated[0] - temp_config['first_domain_temperature_text']) > 0.02:
    print(f"1. Temperature {temp_config['first_domain_temperature_text']:.2f} → {best_integrated[0]:.2f}로 변경")
    print(f"   예상 성능 향상: {best_integrated[1]['accuracy']:.1%}")

print("\n🎯 다음 단계:")
print("1. Temperature 수정 후 재실험")
print("2. Projection Layer 상세 분석")
print("3. InfoNCE Loss 구현 검토")
print("4. 전체 모델 재학습 고려")



🎯 Text-Vibration 정렬 문제 진단 결과
📊 성능 요약:
  개별 Text Encoder: 83.0% (이전 테스트)
  개별 Vibration Encoder: 88.6% (이전 테스트)
  TextVibCLIP 통합: 14.4% (실제 성능)
  성능 손실: 157.2%p (심각한 정렬 문제)

🔍 정렬 분석 결과:
  개별 Encoder 정렬 점수: 0.000090
  통합 모델 정렬 점수: -0.000022
  정렬 점수 차이: 0.000112

🌡️ Temperature 최적화:
  개별 최적 Temperature: 0.01
  통합 최적 Temperature: 0.01
  현재 설정값: 0.05

🚨 문제 진단:
  ❌ 심각한 정렬 문제: 통합 과정에서 정렬이 크게 악화됨
  원인: Projection Layer, InfoNCE Loss, 또는 모델 아키텍처 문제

💡 해결 방안 우선순위:
1. 🔥 Temperature 최적화
   - 현재: 0.05
   - 권장: 0.01
2. 🎯 Projection Layer 점검
   - 차원 정렬 확인
   - 정규화 방법 검토
3. 📐 InfoNCE Loss 구현 검토
   - 유사도 계산 로직
   - Negative sampling 방식
4. 🔧 모델 아키텍처 개선
   - Cross-attention 메커니즘 추가
   - Alignment regularization

⚡ 즉시 수정 가능한 사항:
1. Temperature 0.05 → 0.01로 변경
   예상 성능 향상: 0.3%

🎯 다음 단계:
1. Temperature 수정 후 재실험
2. Projection Layer 상세 분석
3. InfoNCE Loss 구현 검토
4. 전체 모델 재학습 고려


## 7. 긴급 수정: 가중치 동기화 테스트


In [9]:
# 🚨 긴급 수정: 통합 모델에 개별 모델 가중치 복사
print("\n🚨 가중치 동기화 테스트")
print("=" * 50)

# 1. 개별 모델 가중치를 통합 모델에 복사
print("1. Text Encoder 가중치 복사...")
textvib_model.text_encoder.load_state_dict(text_encoder.state_dict())

print("2. Vibration Encoder 가중치 복사...")
textvib_model.vibration_encoder.load_state_dict(vibration_encoder.state_dict())

print("✅ 가중치 복사 완료")

# 2. 가중치 동기화 후 임베딩 재생성
print("\n3. 동기화 후 임베딩 재생성...")
with torch.no_grad():
    # 통합 모델 임베딩 재생성 (가중치 동기화 후)
    text_embeddings_synced = []
    vib_embeddings_synced = []
    
    for i in range(0, len(all_texts), batch_size):
        end_idx = min(i + batch_size, len(all_texts))
        batch_texts = all_texts[i:end_idx]
        batch_vibs = vibration_tensor[i:end_idx].to(device)
        
        # 동기화된 encoder로 임베딩 생성
        batch_text_emb = textvib_model.text_encoder.encode_texts(batch_texts, device)
        batch_vib_emb = textvib_model.vibration_encoder(batch_vibs)
        
        # 정규화
        batch_text_emb = F.normalize(batch_text_emb, p=2, dim=1)
        batch_vib_emb = F.normalize(batch_vib_emb, p=2, dim=1)
        
        text_embeddings_synced.append(batch_text_emb.cpu())
        vib_embeddings_synced.append(batch_vib_emb.cpu())
    
    text_embeddings_synced = torch.cat(text_embeddings_synced, dim=0)
    vib_embeddings_synced = torch.cat(vib_embeddings_synced, dim=0)

# 3. 동기화 후 차이 확인
text_diff_synced = torch.norm(text_embeddings_individual.cpu() - text_embeddings_synced.cpu()).item()
vib_diff_synced = torch.norm(vib_embeddings_individual.cpu() - vib_embeddings_synced.cpu()).item()

print(f"✅ 동기화 후 차이:")
print(f"  Text 차이: {text_diff_synced:.6f}")
print(f"  Vibration 차이: {vib_diff_synced:.6f}")

if text_diff_synced < 1e-6 and vib_diff_synced < 1e-6:
    print("🎯 완벽한 동기화! 가중치 문제였음")
else:
    print("⚠️ 여전히 차이 존재 - 다른 원인 탐색 필요")

# 4. 동기화 후 InfoNCE 성능 테스트
print("\n4. 동기화 후 InfoNCE 성능:")
synced_result = calculate_infonce_loss(
    text_embeddings_synced,
    vib_embeddings_synced, 
    temperature=0.05
)
print(f"  Loss: {synced_result['loss']:.4f}")
print(f"  Accuracy: {synced_result['accuracy']:.4f} ({synced_result['accuracy']*100:.1f}%)")

if synced_result['accuracy'] > 0.1:
    print("🎉 가중치 동기화로 문제 해결!")
else:
    print("🔍 가중치 외 다른 문제 존재")



🚨 가중치 동기화 테스트
1. Text Encoder 가중치 복사...
2. Vibration Encoder 가중치 복사...
✅ 가중치 복사 완료

3. 동기화 후 임베딩 재생성...
✅ 동기화 후 차이:
  Text 차이: 0.000016
  Vibration 차이: 0.000000
⚠️ 여전히 차이 존재 - 다른 원인 탐색 필요

4. 동기화 후 InfoNCE 성능:
  Loss: 5.8602
  Accuracy: 0.0029 (0.3%)
🔍 가중치 외 다른 문제 존재


## 8. InfoNCE Loss 구현 문제 분석 및 수정


In [10]:
# 🔥 InfoNCE Loss 구현 문제 분석 및 수정
print("\n🔥 InfoNCE Loss 구현 문제 진단")
print("=" * 60)

def improved_infonce_loss(text_emb, vib_emb, labels, temperature=0.07):
    """개선된 InfoNCE Loss - 클래스 기반 정확한 구현"""
    text_emb_cpu = text_emb.cpu()
    vib_emb_cpu = vib_emb.cpu()
    labels_cpu = labels.cpu()
    
    # 유사도 행렬 계산
    similarity_matrix = torch.matmul(text_emb_cpu, vib_emb_cpu.t()) / temperature
    
    # 🎯 핵심 수정: 클래스 기반 positive pair 정의
    # 같은 인덱스가 아니라 같은 클래스끼리 positive
    batch_size = len(text_emb_cpu)
    positive_mask = (labels_cpu.unsqueeze(0) == labels_cpu.unsqueeze(1)).float()
    
    # InfoNCE Loss 계산 (클래스 기반)
    # 각 text에 대해 같은 클래스의 vibration들이 positive
    losses = []
    correct_predictions = 0
    
    for i in range(batch_size):
        # i번째 text와 모든 vibration의 유사도
        logits = similarity_matrix[i]  # (batch_size,)
        
        # 같은 클래스 마스크
        pos_mask = positive_mask[i]  # (batch_size,)
        
        if pos_mask.sum() > 0:  # positive가 존재하는 경우만
            # Positive의 평균 유사도
            pos_logits = logits[pos_mask.bool()]
            pos_mean = pos_logits.mean()
            
            # 전체 logits에 대한 logsumexp
            log_sum_exp = torch.logsumexp(logits, dim=0)
            
            # InfoNCE loss
            loss = log_sum_exp - pos_mean
            losses.append(loss)
            
            # 예측: 가장 높은 유사도를 가진 vibration
            pred_idx = torch.argmax(logits)
            if pos_mask[pred_idx] > 0:
                correct_predictions += 1
    
    avg_loss = torch.stack(losses).mean() if losses else torch.tensor(float('inf'))
    accuracy = correct_predictions / batch_size
    
    return {
        'loss': avg_loss.item(),
        'accuracy': accuracy,
        'similarity_matrix': similarity_matrix,
        'positive_mask': positive_mask
    }

# 🎯 개선된 InfoNCE Loss로 테스트
print("1. 개선된 InfoNCE Loss 테스트:")
improved_result = improved_infonce_loss(
    text_embeddings_synced,
    vib_embeddings_synced,
    labels_tensor,
    temperature=0.07
)

print(f"  개선된 Loss: {improved_result['loss']:.4f}")
print(f"  개선된 Accuracy: {improved_result['accuracy']:.4f} ({improved_result['accuracy']*100:.1f}%)")

# 2. 다양한 Temperature로 개선된 Loss 테스트
print(f"\n2. 개선된 InfoNCE - Temperature별 성능:")
print("Temp\tLoss\tAccuracy")
print("-" * 30)

best_temp = 0.07
best_acc = 0.0

for temp in [0.01, 0.03, 0.05, 0.07, 0.1, 0.2, 0.5, 1.0]:
    result = improved_infonce_loss(
        text_embeddings_synced,
        vib_embeddings_synced,
        labels_tensor,
        temperature=temp
    )
    print(f"{temp:.2f}\t{result['loss']:.4f}\t{result['accuracy']:.4f}")
    
    if result['accuracy'] > best_acc:
        best_acc = result['accuracy']
        best_temp = temp

print(f"\n🎯 최적 설정:")
print(f"  최적 Temperature: {best_temp:.2f}")
print(f"  최고 정확도: {best_acc:.4f} ({best_acc*100:.1f}%)")

if best_acc > 0.5:
    print("🎉 InfoNCE Loss 문제 해결!")
elif best_acc > 0.2:
    print("⚠️ 부분적 개선 - 추가 조정 필요")
else:
    print("❌ 여전히 심각한 문제 - 근본적 재설계 필요")



🔥 InfoNCE Loss 구현 문제 진단
1. 개선된 InfoNCE Loss 테스트:
  개선된 Loss: 5.8589
  개선된 Accuracy: 0.1429 (14.3%)

2. 개선된 InfoNCE - Temperature별 성능:
Temp	Loss	Accuracy
------------------------------
0.01	5.9511	0.1429
0.03	5.8667	0.1429
0.05	5.8605	0.1429
0.07	5.8589	0.1429
0.10	5.8582	0.1429
0.20	5.8578	0.1429
0.50	5.8578	0.1429
1.00	5.8579	0.1429

🎯 최적 설정:
  최적 Temperature: 0.01
  최고 정확도: 0.1429 (14.3%)
❌ 여전히 심각한 문제 - 근본적 재설계 필요


## 9. 근본적 해결책: Cross-Modal Projection 테스트


In [11]:
# 🎯 근본적 해결책: Cross-Modal Projection Layer 추가
print("\n🎯 Cross-Modal Projection 테스트")
print("=" * 60)

class CrossModalProjection(nn.Module):
    """Cross-Modal Projection Layer - 두 임베딩 공간을 공통 공간으로 매핑"""
    def __init__(self, input_dim=256, output_dim=128):
        super().__init__()
        self.text_proj = nn.Sequential(
            nn.Linear(input_dim, output_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(output_dim * 2, output_dim),
            nn.LayerNorm(output_dim)
        )
        self.vib_proj = nn.Sequential(
            nn.Linear(input_dim, output_dim * 2),
            nn.ReLU(), 
            nn.Dropout(0.1),
            nn.Linear(output_dim * 2, output_dim),
            nn.LayerNorm(output_dim)
        )
    
    def forward(self, text_emb, vib_emb):
        text_proj = F.normalize(self.text_proj(text_emb), p=2, dim=1)
        vib_proj = F.normalize(self.vib_proj(vib_emb), p=2, dim=1)
        return text_proj, vib_proj

# Cross-Modal Projection 생성 및 간단한 학습
projection = CrossModalProjection(input_dim=256, output_dim=128).to(device)
optimizer = torch.optim.Adam(projection.parameters(), lr=0.001)

print("1. Cross-Modal Projection 학습 (10 epochs)...")

# 간단한 학습 루프
projection.train()
for epoch in range(10):
    total_loss = 0
    num_batches = 0
    
    for i in range(0, len(text_embeddings_synced), 32):
        end_idx = min(i + 32, len(text_embeddings_synced))
        batch_text = text_embeddings_synced[i:end_idx].to(device)
        batch_vib = vib_embeddings_synced[i:end_idx].to(device)
        batch_labels = labels_tensor[i:end_idx].to(device)
        
        # Projection 적용
        text_proj, vib_proj = projection(batch_text, batch_vib)
        
        # InfoNCE Loss (간단한 대각선 버전)
        similarity = torch.matmul(text_proj, vib_proj.t()) / 0.1
        labels_batch = torch.arange(len(batch_text)).to(device)
        loss = F.cross_entropy(similarity, labels_batch)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    if epoch % 2 == 0:
        print(f"  Epoch {epoch}: Loss = {total_loss/num_batches:.4f}")

projection.eval()

# 2. 학습된 Projection으로 임베딩 변환
print("\n2. 학습된 Projection 적용...")
with torch.no_grad():
    text_projected = []
    vib_projected = []
    
    for i in range(0, len(text_embeddings_synced), 32):
        end_idx = min(i + 32, len(text_embeddings_synced))
        batch_text = text_embeddings_synced[i:end_idx].to(device)
        batch_vib = vib_embeddings_synced[i:end_idx].to(device)
        
        text_proj, vib_proj = projection(batch_text, batch_vib)
        text_projected.append(text_proj.cpu())
        vib_projected.append(vib_proj.cpu())
    
    text_projected = torch.cat(text_projected, dim=0)
    vib_projected = torch.cat(vib_projected, dim=0)

print(f"✅ Projection 적용 완료:")
print(f"  변환 전: {text_embeddings_synced.shape[1]}차원")
print(f"  변환 후: {text_projected.shape[1]}차원")

# 3. Projection 후 InfoNCE 성능 테스트
print("\n3. Projection 후 InfoNCE 성능:")
projected_result = improved_infonce_loss(
    text_projected,
    vib_projected,
    labels_tensor,
    temperature=0.1
)

print(f"  Projection 후 Loss: {projected_result['loss']:.4f}")
print(f"  Projection 후 Accuracy: {projected_result['accuracy']:.4f} ({projected_result['accuracy']*100:.1f}%)")

# 4. 성능 비교 요약
print(f"\n📊 최종 성능 비교:")
print(f"  개별 Text Encoder: 83.0%")
print(f"  개별 Vibration Encoder: 88.6%")
print(f"  기존 통합 (InfoNCE): 0.3%")
print(f"  Projection 후: {projected_result['accuracy']*100:.1f}%")

improvement = projected_result['accuracy'] * 100 - 0.3
print(f"  개선도: +{improvement:.1f}%p")

if projected_result['accuracy'] > 0.5:
    print("🎉 Cross-Modal Projection으로 문제 해결!")
elif projected_result['accuracy'] > 0.3:
    print("⚠️ 부분적 개선 - Projection 아키텍처 조정 필요")
else:
    print("❌ 근본적 문제 - 다른 접근법 필요")



🎯 Cross-Modal Projection 테스트
1. Cross-Modal Projection 학습 (10 epochs)...
  Epoch 0: Loss = 3.4657
  Epoch 2: Loss = 3.4604
  Epoch 4: Loss = 3.4628
  Epoch 6: Loss = 3.4574
  Epoch 8: Loss = 3.4597

2. 학습된 Projection 적용...
✅ Projection 적용 완료:
  변환 전: 256차원
  변환 후: 128차원

3. Projection 후 InfoNCE 성능:
  Projection 후 Loss: 5.8502
  Projection 후 Accuracy: 0.2857 (28.6%)

📊 최종 성능 비교:
  개별 Text Encoder: 83.0%
  개별 Vibration Encoder: 88.6%
  기존 통합 (InfoNCE): 0.3%
  Projection 후: 28.6%
  개선도: +28.3%p
❌ 근본적 문제 - 다른 접근법 필요


## 10. 최종 진단 및 실제 해결책


In [12]:
print("\n" + "="*80)
print("🎯 TextVibCLIP 정렬 문제 최종 진단 및 해결책")
print("="*80)

print(f"📊 문제 진단 요약:")
print(f"1. ✅ Text Encoder 단독: 83.0% (우수)")
print(f"2. ✅ Vibration Encoder 단독: 88.6% (우수)")
print(f"3. ❌ 가중치 불일치: 해결됨 (차이 0.000016)")
print(f"4. ❌ InfoNCE Loss 구현: 14.3% (랜덤 수준)")
print(f"5. 🎯 Cross-Modal Projection: {projected_result['accuracy']*100:.1f}%")

print(f"\n🔍 근본 원인:")
print(f"Text와 Vibration 임베딩이 완전히 다른 의미 공간에 존재")
print(f"- Text: 언어적 의미 공간 (단어, 문법 기반)")
print(f"- Vibration: 물리적 신호 공간 (주파수, 진폭 기반)")
print(f"- 결과: 두 공간 간 자연스러운 매핑이 존재하지 않음")

print(f"\n💡 해결책 우선순위:")
if projected_result['accuracy'] > 0.6:
    print("1. 🎉 Cross-Modal Projection 성공 - 이 방식 적용")
    print("2. TextVibCLIP 모델에 Projection Layer 통합")
    print("3. Projection Layer 파라미터 최적화")
elif projected_result['accuracy'] > 0.3:
    print("1. ⚠️ Projection 부분 성공 - 아키텍처 개선")
    print("2. Projection Layer 깊이/너비 조정")
    print("3. 다른 정렬 방법 병행 (Attention, CCA 등)")
else:
    print("1. ❌ 근본적 재설계 필요")
    print("2. 사전 학습된 멀티모달 모델 활용 고려")
    print("3. 대안: 각 Encoder 독립 사용 + 앙상블")

print(f"\n🚀 즉시 적용 방안:")
print(f"1. TextVibCLIP 모델에 Cross-Modal Projection 추가")
print(f"2. Temperature 최적화: 0.05 → 0.1")
print(f"3. Projection 학습률 및 아키텍처 조정")

print(f"\n📈 예상 성능 (Projection 적용 시):")
expected_performance = min(80, max(30, projected_result['accuracy'] * 100))
print(f"  TextVibCLIP 통합: 14.4% → {expected_performance:.0f}%")
print(f"  성능 회복: {expected_performance - 14.4:.1f}%p")

if projected_result['accuracy'] > 0.4:
    print("\n🎯 다음 단계: TextVibCLIP 모델 수정")
    print("1. src/textvib_model.py에 CrossModalProjection 추가")
    print("2. Forward pass에서 Projection 적용")
    print("3. 전체 모델 재학습")
else:
    print("\n🔍 다음 단계: 대안 접근법 고려")
    print("1. 사전 학습된 멀티모달 모델 활용")
    print("2. 각 Encoder 독립 사용 + Late Fusion")
    print("3. Domain-specific Adapter 추가")



🎯 TextVibCLIP 정렬 문제 최종 진단 및 해결책
📊 문제 진단 요약:
1. ✅ Text Encoder 단독: 83.0% (우수)
2. ✅ Vibration Encoder 단독: 88.6% (우수)
3. ❌ 가중치 불일치: 해결됨 (차이 0.000016)
4. ❌ InfoNCE Loss 구현: 14.3% (랜덤 수준)
5. 🎯 Cross-Modal Projection: 28.6%

🔍 근본 원인:
Text와 Vibration 임베딩이 완전히 다른 의미 공간에 존재
- Text: 언어적 의미 공간 (단어, 문법 기반)
- Vibration: 물리적 신호 공간 (주파수, 진폭 기반)
- 결과: 두 공간 간 자연스러운 매핑이 존재하지 않음

💡 해결책 우선순위:
1. ❌ 근본적 재설계 필요
2. 사전 학습된 멀티모달 모델 활용 고려
3. 대안: 각 Encoder 독립 사용 + 앙상블

🚀 즉시 적용 방안:
1. TextVibCLIP 모델에 Cross-Modal Projection 추가
2. Temperature 최적화: 0.05 → 0.1
3. Projection 학습률 및 아키텍처 조정

📈 예상 성능 (Projection 적용 시):
  TextVibCLIP 통합: 14.4% → 30%
  성능 회복: 15.6%p

🔍 다음 단계: 대안 접근법 고려
1. 사전 학습된 멀티모달 모델 활용
2. 각 Encoder 독립 사용 + Late Fusion
3. Domain-specific Adapter 추가
